# Workflow Examples 


`xclim` is built on very powerful multiprocessing and distributed computation libraries, notably `xarray` and `dask`.

`xarray` is a python package making it easy to work with n-dimensional arrays. It labels axes with their names `[time, lat, lon, level]` instead of indices `[0,1,2,3]`, reducing the likelihood of bugs and making the code easier to understand. One of the key strengths of `xarray` is that it knows how to deal with non-standard calendars (we're looking at you, "`360_days`") and can easily resample daily time series to weekly, monthly, seasonal or annual periods.  Finally, `xarray` is tightly inegrated with `dask`, a package that can automatically parallelize operations.

The following are a few examples to consult when using `xclim` to subset netCDF arrays and compute climate indicators, taking advantage of the parallel processing capabilities offered by `xarray` and `dask`. For more information about these projects, please see their documentation pages:

* [xarray documentation](https://xarray.pydata.org/en/stable/)
* [dask documentation](https://docs.dask.org/en/stable/)

## Environment configuration

In [ ]:
# Imports for xclim and xarray
import xclim as xc
import numpy as np
import xarray as xr
xr.set_options(display_style='html')

# plotting
import matplotlib.pyplot as plt
%matplotlib inline

# File handling libraries
import time
import tempfile
from pathlib import Path

# Output folder
output_folder = Path(tempfile.mkdtemp()) 

## Setting up the Dask client: parallel processing

First we create a pool of workers that will wait for jobs. The `xarray` library will automatically connect to these workers and and dispatch them jobs that can be run in parallel.  

The dashboard link lets you see in real time how busy those workers are.

* [dask distributed documentation](https://distributed.dask.org/en/latest/)

This step is not mandatory as dask will fall back to its "single machine scheduler" if a `Client` is not created. However, this default scheduler doesn't allow you to set the number of threads or a memory limit and doesn't start the dashboard, which can be quite useful to understand your task's progress.


In [ ]:
from distributed import Client

# Depending on your workstation specifications, you may need to adjust these values.
# On a single machine, n_workers=1 is usually better.
client=Client(n_workers=1, threads_per_worker=4, memory_limit="4GB") 
client

## Creating xarray datasets

To open a netCDF file with `xarray`, we use `xr.open_dataset(<path to file>)`. By default, the entire file is stored in one chunk, so there is no parallelism. To trigger parallel computations, we need to explicitly specify the **chunk size**. 

<div class="alert alert-info">

In this example, instead of opening a local file, we pass an *OPeNDAP* url to xarray. It retrieves the data automatically. Notice also that opening the dataset is quite fast. In fact, the data itself has not been downloaded yet, only the coordinates and the metadata. The downloads will be triggered only when the values need to be accessed directly.

</div>

`dask`'s parallelism is based on memory chunks. We need to tell `xarray` to split our netCDF array into chunks of a given size, and operations on each chunk of the array will automatically be dispatched to the workers. 

In [ ]:
data_url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/cb-oura-1.0/ACCESS1-3/rcp85/day/ACCESS1-3_rcp85_day_allvars.ncml'

In [ ]:
# Chunking in memory along the time dimension.
# Note that the data type is a 'dask.array'. xarray will automatically use client workers.
ds = xr.open_dataset(data_url, chunks={'time': 365, 'lat': 168, 'lon': 150}, drop_variables=['ts', 'time_vectors'])
ds

In [ ]:
print(ds.tasmin.chunks)

## Multi-file datasets

NetCDF files are often split into periods to keep file size manageable. A single dataset can be split in dozens of individual files. `xarray` has a function `open_mfdataset` that can open and aggregate a list of files and construct a unique *logical* dataset. `open_mfdataset` can aggregate files over coordinates (time, lat, lon) and variables. 

* Note that opening a multi-file dataset automatically chunks the array (one chunk per file).
* Note also that because `xarray` reads every file metadata to place it in a logical order, it can take a while to load. 

In [ ]:
## Create multi-file data & chunks 
# ds = xr.open_mfdataset('/path/to/files*.nc')

## Subsetting utilities

`xclim` comes with some utilities to perform common tasks that are either not implemented in xarray, or that are implemented but do not have the generality needed for climate science work.

### subset_bbox : using a latitude-longitude bounding box

In [ ]:
from xclim import subset
lat_bnds = [45, 50]
lon_bnds = [-60, -55]

ds1 = subset.subset_bbox(ds, lat_bnds=lat_bnds, lon_bnds=lon_bnds)
ds1.tasmax

### Add time subsetting

In [ ]:
ds2 = subset.subset_bbox(ds, lat_bnds=lat_bnds, lon_bnds=lon_bnds,
                         start_date='2015-01', end_date='2030-05')
ds2.tasmax

### Selecting a single grid point 
`subset_gridpoint` can be used for selecting single locations. As other function of this submodule, it mostly replicated the behavior of `xarray.DataArray.sel()`. In this case, it adds a `tolerance` parameter so that it finds the nearest point from the given coordinate within this distance, or else it raises an error.

In [ ]:
lon_pt = -70.0
lat_pt = 50.0

ds3 = subset.subset_gridpoint(ds, lon=lon_pt, lat=lat_pt, tolerance=10000,
                              start_date='1993', end_date='2020')
ds3.tasmax

#### Create a plot  of subsetted data
xarray provide a very simple plotting interface to easily explore our data.

*(While not doing any computation, this operation needs to download the data first.)*

In [ ]:
ds3.tasmax.plot()

## Climate index calculation & resampling frequencies

`xclim` has two layers for the calculation of indicators. The bottom layer is composed of a list of functions that take one or more `xarray.DataArray`'s as input and return an `xarray.DataArray` as output. You'll find these functions in `xclim.indices`. The indicator's logic is contained in this function, as well as potential unit conversions, but it doesn't perform any data consistency checks (like if the time frequency is daily), and doesn't not adjust the meta data of the output array. 

The second layer are class instances that you'll find organized by *realm*. So far, there are three realms available in `xclim.atmos`, `xclim.seaIce` and `xclim.land`, the first one being the most exhaustive. Before running computations, these classes check if the input data is a daily average of the expected variable:

1. If an indicator expects a daily mean and you pass it a daily max, a `warning` will be raised. 
2. After the computation, it also checks the number of values per period to make sure there are not missing values or `NaN` in the input data. If there are, the output is going to be set to `NaN`.  Ex. : If the indicator performs a yearly resampling but there are only 350 non-`NaN` values in one given year in the input data, that year's output will be `NaN`.
3. The output units are set correctly as well as other properties of the output array, complying as much as possible with CF conventions. 

For new users, we suggest you use the classes found in `xclim.atmos` and others. If you know what you're doing and you want to circumvent the built-in checks, then you can use the `xclim.indices` directly. 

Almost all `xclim` indicators convert daily data to lower time frequencies, such as seasonal or annual values. This is done using `xarray.DataArray.resample` method. Resampling creates a grouped object over which you apply a reduction operation (e.g. mean, min, max). The list of available frequency is given in the link below, but the most often used are: 

- `YS`: annual starting in January
- `YS-JUL`: annual starting in July
- `MS`: monthly
- `QS-DEC`: seasonal starting in December

More info about this specification can be found in [pandas' documentation](http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases)

Note - not all offsets in the link are supported by cftime objects in `xarray`.


In the example below, we're computing the **annual maximum temperature of the daily maximum temperature (tx_max)**.

In [ ]:
out = xc.atmos.tx_max(ds2.tasmax, freq='YS')
out

<div class="alert alert-info">

If you execute the cell above, you'll see that this operation is quite fast. This a feature coming from dask. Read *Lazy computation*  further down.

</div>

### Comparison of **atmos** vs **indices** modules


In [ ]:
out = xc.indices.tx_days_above(ds3.tasmax,thresh='30 C',freq='YS')
print('Example output using indices module : \n')
out

In [ ]:
out = xc.atmos.tx_days_above(ds2.tasmax,thresh='30 C',freq='YS')
print('Example output using atmos module : Output data has added CF metadata attributes\n\n')
out

In [ ]:
# We have created an xarray data-array - We can insert this into an output xr.Dataset object with a copy of the original dataset global attrs
dsOut = xr.Dataset(attrs=ds2.attrs)

# Add our climate index as a data variable to the dataset
dsOut[out.name] = out
dsOut

## *Lazy* computation - Nothing has been computed so far !

If you look at the output of those operations, they're identified as `dask.array` objects. What happens is that `dask` creates a chain of operations that when executed, will yield the values we want. We have thus far only created a schedule of tasks with a small preview and not done any actual computations. You can trigger computations by using the `load` or `compute` method, or writing the output to disk via `to_netcdf`. Of course, calling `.plot()` will also trigger the computation.

In [ ]:
%%time
output_file = output_folder / 'test_tx_max.nc'
dsOut.to_netcdf(output_file)

*(Times may of course vary depending on the machine and the Client settings)*

### Performance tips
#### Optimizing the chunk size

You can improve performance by being smart about chunk sizes. If chunks are too small, there is a lot of time lost in overhead. If chunks are too large, you may end up exceeding the individual worker memory limit.

In [ ]:
print(ds3.chunks['time'])

In [ ]:
# rechunk data in memory for the entire grid  
ds3c = ds3.chunk(chunks={'time':4 * 365})
print(ds3c.chunks['time'])

In [ ]:
%%time
out = xc.atmos.tx_max(ds3c.tasmax, freq='YS')
dsOut = xr.Dataset(data_vars=None, coords=out.coords, attrs=ds1.attrs)
dsOut[out.name] = out

output_file = output_folder / 'test_tx_max.nc'
dsOut.to_netcdf(output_file)

## Unit handling in `xclim`

A lot of effort has been placed into automatic handling of input data units. `xclim` will automatically detect the input variable(s) units (e.g. °C versus °K or mm/s versus mm/day etc.) and adjust on-the-fly in order to calculate indices in the consistent manner. This comes with the obvious caveat that input data requires metadata attribute for units

In the example below, we compute weekly total precipitation in mm using inputs of mm/s and mm/d:

In [ ]:
# Create a copy of the data converted to mm d-1
pr_mmd = ds2.pr * 3600 * 24
pr_mmd.attrs["units"] = "mm d-1"

In [ ]:
out1 = xc.atmos.precip_accumulation(ds2.pr, freq='MS')
print('1. results using inputs in mm/s :')
out1

In [ ]:
out2 = xc.atmos.precip_accumulation(pr_mmd, freq='MS')
print("2. results using inputs in mm/d :")
out2   

### Threshold indices

`xclim` unit handling also applies to threshold indicators. Users can provide threshold in units of choice and `xclim` will adjust automatically. For example determining the number of days with tasmax > 20°C users can define a threshold input of '20 C' or '20 degC' even if input data is in Kelvin. Alernatively users can even provide a threshold in Kelvin '293.15 K' (if they really wanted to).

In [ ]:
# Create a copy of the data converted to C
tasmax_C = ds3.tasmax - 273.15
tasmax_C.attrs['units'] = 'C'

In [ ]:
# Using Kelvin data, threshold in Celsius
out1 = xc.atmos.tx_days_above(ds3.tasmax, thresh='20 C', freq='MS')
print('1. results using inputs in °K  : threshold in °C:')
out1

In [ ]:
# Using Celsius data
out2 = xc.atmos.tx_days_above(tasmax_C, thresh='20 C', freq='MS')
print('2. results using inputs in °C : threshold in °C')
out2

In [ ]:
# Using Celsius but with threshold in Kelvin
out3 = xc.atmos.tx_days_above(tasmax_C, thresh='293.15 K', freq='MS')
print('\n3. results using inputs in °C : threshold in °K :')
out3